# MF-DGP-EM

In [1]:
import tensorflow as tf
import gpflow
import numpy as np
import emukit.multi_fidelity
import emukit.test_functions
from dgp_dace.models.MF_DGP_EM import MultiFidelityDeepGP_EM
from gpflow import set_trainable
from sklearn.metrics import mean_squared_error, r2_score
import scipy
from prettytable import PrettyTable
import pyDOE

In [2]:
from emukit.test_functions.multi_fidelity import (multi_fidelity_borehole_function, multi_fidelity_branin_function,
                                                  multi_fidelity_park_function, multi_fidelity_hartmann_3d,
                                                  multi_fidelity_currin_function)
from emukit.core import ContinuousParameter, ParameterSpace
from emukit.experimental_design.model_free.latin_design import LatinDesign

In [3]:
from emukit.core.loop.user_function import MultiSourceFunctionWrapper
from typing import Tuple
from collections import namedtuple
from emukit.core import ParameterSpace, ContinuousParameter, InformationSourceParameter
Function = namedtuple('Function', ['name', 'y_scale', 'noise_level', 'do_x_scaling', 'num_data', 'fcn','input_dim'])


In [4]:
def multi_fidelity_park_VD() -> Tuple[MultiSourceFunctionWrapper, ParameterSpace]:
    def high(x):
        x1 = x[:, 0]
        x2 = x[:, 2]
        x3 = x[:, 1]
        x4 = x[:, 3]
        tmp = 1 + (x2 + x3) * (x4 / x1**2)
        return ((x1 / 2) * (np.sqrt(tmp) - 1) + (x1 + 3 * x4) * np.exp(1 + np.sin(x3)))[:, None]
    def low(x):
        # f_high = high(x)
        f_high = high(np.concatenate((x,0.5*np.ones((x.shape[0],2))),1))
        x1 = x[:, 0]
        x2 = x[:, 1]
        # x3 = x[:, 2]
        # x4 = x[:, 3]
        return ((1 + np.sin(x1) / 10) * f_high.flatten() - 2 * x1 + x2**2 + 0.5**2 + 0.5)[:, None]#f_high.flatten()[:, None] #
    space = ParameterSpace([ContinuousParameter('x1', 0, 1), ContinuousParameter('x2', 0, 1),
                            InformationSourceParameter(2)])
    return MultiSourceFunctionWrapper([low, high]), space

In [5]:
Park_VD = Function(name='Park_VD', y_scale=1, noise_level=[0., 0.], do_x_scaling=False, num_data=[30, 6], 
                fcn=multi_fidelity_park_VD, input_dim=[2,4])      

In [6]:

def generate_data(fcn_tuple,sd=100):
    """
    Generates train and test data for
    """
    # A different definition of the parameter space for the branin function was used in the paper
    if fcn_tuple.name == 'branin':
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace([ContinuousParameter('x1', -5., 0.), ContinuousParameter('x2', 10., 15.)])
    else:
        fcn, space = fcn_tuple.fcn()
        new_space = ParameterSpace(space._parameters[:-1])
    do_x_scaling = fcn_tuple.do_x_scaling
    # Generate training data
    X = [pyDOE.lhs(d,n,seed=sd_) for n,d,sd_ in zip(fcn_tuple.num_data,fcn_tuple.input_dim,[123,sd])]
    # Scale X if required
#    if do_x_scaling:
#        scalings = X[0].std(axis=0)
#    else:
#        scalings = np.ones(X[0].shape[1])
#        
#    for x in X:
#        x /= scalings
    Y = []
    for i, x in enumerate(X):
        Y.append(fcn.f[i](x))
    # scale y and add noise if required
    noise_levels = fcn_tuple.noise_level
#    for y, std_noise in zip(Y, noise_levels):
#        y /= y_scale + std_noise * np.random.randn(y.shape[0], 1)
    # Generate test data
    x_test = pyDOE.lhs(fcn_tuple.input_dim[-1],1000)
#    x_test /= scalings
    y_test = fcn.f[-1](x_test )
    # y_test = (y_test-mean_y)/std_y
#     if normalize:
#          x_test = (x_test-mean_x)/std_x
#    y_test /= y_scale
    print(X[1].shape)
    return x_test, y_test, X, Y

In [7]:
def calculate_metrics(y_test, y_mean_prediction, y_var_prediction):
    # R2
    r2 = r2_score(y_test, y_mean_prediction)
    # RMSE
    rmse = np.sqrt(mean_squared_error(y_test, y_mean_prediction))
    # Test log likelihood
    mnll = -np.sum(scipy.stats.norm.logpdf(y_test, loc=y_mean_prediction, scale=np.sqrt(y_var_prediction)))/len(y_test)
    return {'r2': r2, 'rmse': rmse, 'mnll': mnll}


In [8]:
sd = 0
x_test, y_test, X, Y= generate_data(Park_VD,sd=sd)
X_red = X[1][:,:2]

(6, 4)


In [18]:
model = MultiFidelityDeepGP_EM(X, Y,[X_red])
model.optimize_nat_adam(lr_adam = 0.01,lr_gamma = 0.01,iterations1=0,iterations2=3000 ,iterations3=15000) ### S=100

ELBO: -1381.3228927543003
ELBO: -937.3176678832966
ELBO: -715.1279544868934
ELBO: -564.5640427243848
ELBO: -465.23674958154663
ELBO: -398.39880197489555
ELBO: -352.8819677081529
ELBO: -144.86187182928697
ELBO: -66.31050796128063
ELBO: -32.48282900260296
ELBO: -31.760572485196374
ELBO: -24.106293526939993
ELBO: -358.3613157178246
ELBO: -12.8562308381367
ELBO: -8.023387547618992
ELBO: -5.8533572326578245
ELBO: -33.42981673749546
ELBO: -1.8225778872164895
ELBO: -0.8414113286331286
ELBO: -80.39794519021022
ELBO: 2.443156989734277
ELBO: 3.7116162722612422
ELBO: 4.593490356483031
ELBO: 4.2669281059101785
ELBO: -175.29749907549242
ELBO: 5.445050112682594
ELBO: 5.272104888838886
ELBO: 8.290373312264386
ELBO: 5.969367942497339
ELBO: -11.585176132170723
ELBO: 7.952314106879541
ELBO: 9.017765768529053
ELBO: 10.234389073885339
ELBO: 10.3613103670863
ELBO: -171.44121504685958
ELBO: 12.618764886581275


In [19]:
y_mean, y_var = model.predict(x_test)
metrics = calculate_metrics(y_test, y_mean, y_var)

In [20]:
metrics

{'r2': 0.8926508912793821,
 'rmse': 1.4978208084005984,
 'mnll': 2.078440494205319}

In [21]:
model.model

name,class,transform,prior,trainable,shape,dtype,value
DGP_Base.layers[0].q_mu,Parameter,Identity,,False,"(30, 1)",float64,[[12.11984782...
DGP_Base.layers[0].q_sqrt,Parameter,FillTriangular,,False,"(1, 30, 30)",float64,"[[[5.62796004e-03, 0.00000000e+00, 0.00000000e+00..."
DGP_Base.layers[0].feature.Z,Parameter,Identity,,True,"(30, 2)",float64,"[[0.81460241, 0.65482005..."
DGP_Base.layers[0].kern.kernels[0].variance,Parameter,Softplus,,True,(),float64,49.01285014524215
DGP_Base.layers[0].kern.kernels[0].lengthscales,Parameter,Softplus,,True,"(2,)",float64,[2.61114424 1.34178876]
DGP_Base.layers[0].kern.kernels[1].variance,Parameter,Softplus,,True,(),float64,2.6820426105939277e-05
DGP_Base.layers[1].q_mu,Parameter,Identity,,False,"(6, 1)",float64,[[12.79791697...
DGP_Base.layers[1].q_sqrt,Parameter,FillTriangular,,False,"(1, 6, 6)",float64,"[[[0.19828346, 0., 0...."
DGP_Base.layers[1].feature.Z_left,Parameter,Identity,,True,"(6, 4)",float64,"[[0.99649142, 0.6547287, 0.23923421..."
DGP_Base.layers[1].kern.kernels[0].kernels[0].variance,Parameter,Softplus,,True,(),float64,7.7536402627517145


In [11]:
model.model

name,class,transform,prior,trainable,shape,dtype,value
DGP_Base.layers[0].q_mu,Parameter,Identity,,False,"(30, 1)",float64,[[12.11979739...
DGP_Base.layers[0].q_sqrt,Parameter,FillTriangular,,False,"(1, 30, 30)",float64,"[[[7.32239106e-03, 0.00000000e+00, 0.00000000e+00..."
DGP_Base.layers[0].feature.Z,Parameter,Identity,,True,"(30, 2)",float64,"[[0.85753833, 0.63577056..."
DGP_Base.layers[0].kern.kernels[0].variance,Parameter,Softplus,,True,(),float64,5.1202684282597435
DGP_Base.layers[0].kern.kernels[0].lengthscales,Parameter,Softplus,,True,"(2,)",float64,[1.0549015 0.57052981]
DGP_Base.layers[0].kern.kernels[1].variance,Parameter,Softplus,,True,(),float64,4.5554607608940556e-05
DGP_Base.layers[1].q_mu,Parameter,Identity,,False,"(6, 1)",float64,[[12.79351926...
DGP_Base.layers[1].q_sqrt,Parameter,FillTriangular,,False,"(1, 6, 6)",float64,"[[[0.07884944, 0., 0...."
DGP_Base.layers[1].feature.Z_left,Parameter,Identity,,True,"(6, 4)",float64,"[[1.17791167, 0.62330057, 0.22538032..."
DGP_Base.layers[1].kern.kernels[0].kernels[0].variance,Parameter,Softplus,,True,(),float64,0.23439920055907054
